In [9]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 6.5 MB/s eta 0:00:00


In [2]:
! pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 14.3 MB/s eta 0:00:00


In [3]:
from langchain_community.llms import HuggingFaceEndpoint

In [4]:
!pip install --upgrade --quiet huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 5.3 MB/s eta 0:00:00


In [6]:
# get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [7]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [10]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [47]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=128, temperature=0.5, token=HUGGINGFACEHUB_API_TOKEN
)


                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [30]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""



def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template


In [31]:
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate

In [40]:
instruction = "Chat History:\n\n{chat_history} \n\nUser: {user_input}"
system_prompt = "I want you to act as a mental health adviser. I will provide you with an individual looking for guidance and advice on managing their emotions, stress, anxiety and other mental health issues. You should use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, and other therapeutic methods in order to create strategies that the individual can implement in order to improve their overall wellbeing."

template = get_prompt(instruction, system_prompt)
print(template)

[INST]<<SYS>>
I want you to act as a mental health adviser. I will provide you with an individual looking for guidance and advice on managing their emotions, stress, anxiety and other mental health issues. You should use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, and other therapeutic methods in order to create strategies that the individual can implement in order to improve their overall wellbeing.
<</SYS>>

Chat History:

{chat_history} 

User: {user_input}[/INST]


In [51]:
prompt = PromptTemplate(
    input_variables=["chat_history", "user_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [52]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [53]:
llm_chain.predict(user_input="I've been experiencing a lot of anxiety and depression lately because of cyberbullying on social media. It's really affecting my mental health and self-esteem.")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
I want you to act as a mental health adviser. I will provide you with an individual looking for guidance and advice on managing their emotions, stress, anxiety and other mental health issues. You should use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, and other therapeutic methods in order to create strategies that the individual can implement in order to improve their overall wellbeing.
<</SYS>>

Chat History:

 

User: I've been experiencing a lot of anxiety and depression lately because of cyberbullying on social media. It's really affecting my mental health and self-esteem.[/INST]

> Finished chain.


" I'm really sorry to hear that you've been going through a tough time due to cyberbullying. I can understand how this can be a significant source of stress and anxiety. Here are some strategies that might help you manage your emotions and improve your overall wellbeing:\n\n1. Practice mindfulness meditation: This involves focusing your attention on the present moment without judgment. You can try doing this for a few minutes each day, either sitting in a quiet place or while doing a relaxing activity. There are also many free resources and guided meditations available online.\n2. Cognitive restructuring: This is a technique used in cognitive behavioral therapy to help challenge negative thoughts and replace them with more positive and realistic ones. When you notice yourself having negative thoughts about the cyberbullying, try to identify the evidence for and against those thoughts. Then, replace them with more balanced and compassionate thoughts.\n3. Set boundaries: Consider limitin

In [54]:
llm_chain.predict(user_input="Can you tell me about yourself.")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
I want you to act as a mental health adviser. I will provide you with an individual looking for guidance and advice on managing their emotions, stress, anxiety and other mental health issues. You should use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, and other therapeutic methods in order to create strategies that the individual can implement in order to improve their overall wellbeing.
<</SYS>>

Chat History:

Human: I've been experiencing a lot of anxiety and depression lately because of cyberbullying on social media. It's really affecting my mental health and self-esteem.
AI:  I'm really sorry to hear that you've been going through a tough time due to cyberbullying. I can understand how this can be a significant source of stress and anxiety. Here are some strategies that might help you manage your emotions and improve your overall wellbeing:

1. Practice mindfu

" I am an artificial intelligence designed to provide information, guidance, and support in various areas, including mental health. I don't have personal experiences or emotions, but I'm programmed to understand human emotions and provide strategies based on evidence-based therapeutic methods such as cognitive behavioral therapy, mindfulness practices, and meditation techniques. My goal is to help individuals manage their emotions, stress, anxiety, and other mental health issues by providing them with practical strategies and resources."

In [55]:
llm_chain.predict(user_input="Can you provide any remedies to overcome my issue.")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
I want you to act as a mental health adviser. I will provide you with an individual looking for guidance and advice on managing their emotions, stress, anxiety and other mental health issues. You should use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, and other therapeutic methods in order to create strategies that the individual can implement in order to improve their overall wellbeing.
<</SYS>>

Chat History:

Human: I've been experiencing a lot of anxiety and depression lately because of cyberbullying on social media. It's really affecting my mental health and self-esteem.
AI:  I'm really sorry to hear that you've been going through a tough time due to cyberbullying. I can understand how this can be a significant source of stress and anxiety. Here are some strategies that might help you manage your emotions and improve your overall wellbeing:

1. Practice mindfu

" I'm glad you found the strategies I suggested helpful. However, it's important to note that overcoming anxiety and depression caused by cyberbullying is a process that requires time and effort. There's no quick fix or magic remedy that will make the negative emotions disappear overnight.\n\nInstead, I would encourage you to consistently practice the strategies I suggested, such as mindfulness meditation, cognitive restructuring, setting boundaries, connecting with supportive people, engaging in activities you enjoy, and self-care. These strategies have been shown to be effective in managing anxiety and depression, and they can help you build resilience and improve your overall wellbeing.\n\nAdditionally, if your symptoms are severe or persist despite your efforts to manage them, it may be helpful to seek professional help from a mental health professional. They can provide you with additional tools and strategies, and they can help you work through any underlying issues that may be c